<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preprocessing</a></span><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Preliminary-feature-selection" data-toc-modified-id="Preliminary-feature-selection-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Preliminary feature selection</a></span><ul class="toc-item"><li><span><a href="#Features-to-drop" data-toc-modified-id="Features-to-drop-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Features to drop</a></span></li><li><span><a href="#Features-to-use-(-Features-that-can-be-known-at-the-time-of-loan-issuance)" data-toc-modified-id="Features-to-use-(-Features-that-can-be-known-at-the-time-of-loan-issuance)-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Features to use ( Features that can be known at the time of loan issuance)</a></span></li></ul></li><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Data cleaning</a></span></li></ul></li></ul></div>

# Preprocessing

In [256]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
%matplotlib inline
plt.rcParams['figure.figsize'] = (9, 6)
sns.set(style = "whitegrid")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
warnings.filterwarnings(action="ignore")

## Load Data

In [257]:
accepted = pd.read_csv('accepted_2007_to_2018Q4.csv')

In [258]:
# Parse Years
accepted['year'] = pd.to_datetime(accepted.issue_d).dt.year
# Bin late loans into one group
accepted.loan_status = accepted.loan_status.apply(lambda x: np.where(x == 'Late (31-120 days)','Late',x))
accepted.loan_status = accepted.loan_status.apply(lambda x: np.where(x == 'In Grace Period','Late',x))
accepted.loan_status = accepted.loan_status.apply(lambda x: np.where(x == 'Late (16-30 days)','Late',x))
# Reduce the size of the dataset
accepted = accepted[(accepted.year.isin([2016,2017,2018]))& 
                    accepted.loan_status.isin(['Fully Paid','Charged Off','Late'])]

In [259]:
print(accepted.shape)

(550097, 152)


## Preliminary feature selection

In [260]:
accepted.reindex(sorted(accepted.columns), axis = 1).head()

,acc_now_delinq,acc_open_past_24mths,addr_state,all_util,annual_inc,annual_inc_joint,application_type,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,collection_recovery_fee,collections_12_mths_ex_med,debt_settlement_flag,debt_settlement_flag_date,deferral_term,delinq_2yrs,delinq_amnt,desc,disbursement_method,dti,dti_joint,earliest_cr_line,emp_length,emp_title,fico_range_high,fico_range_low,funded_amnt,funded_amnt_inv,grade,hardship_amount,hardship_dpd,hardship_end_date,hardship_flag,hardship_last_payment_amount,hardship_length,hardship_loan_status,hardship_payoff_balance_amount,hardship_reason,hardship_start_date,hardship_status,hardship_type,home_ownership,id,il_util,initial_list_status,inq_fi,inq_last_12m,inq_last_6mths,installment,int_rate,issue_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,last_pymnt_amnt,last_pymnt_d,loan_amnt,loan_status,max_bal_bc,member_id,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_last_delinq,mths_since_last_major_derog,mths_since_last_record,mths_since_rcnt_il,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,next_pymnt_d,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,open_acc,open_acc_6m,open_act_il,open_il_12m,open_il_24m,open_rv_12m,open_rv_24m,orig_projected_additional_accrued_interest,out_prncp,out_prncp_inv,payment_plan_start_date,pct_tl_nvr_dlq,percent_bc_gt_75,policy_code,pub_rec,pub_rec_bankruptcies,purpose,pymnt_plan,recoveries,revol_bal,revol_bal_joint,revol_util,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_earliest_cr_line,sec_app_fico_range_high,sec_app_fico_range_low,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_mths_since_last_major_derog,sec_app_num_rev_accts,sec_app_open_acc,sec_app_open_act_il,sec_app_revol_util,settlement_amount,settlement_date,settlement_percentage,settlement_status,settlement_term,sub_grade,tax_liens,term,title,tot_coll_amt,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bal_ex_mort,total_bal_il,total_bc_limit,total_cu_tl,total_il_high_credit_limit,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,total_rev_hi_lim,url,verification_status,verification_status_joint,year,zip_code
421101,0.0,3.0,WA,1.0,52000.0,NaN,Individual,25099.0,30359.0,0.5,0.0,0.0,0.0,N,NaN,NaN,0.0,0.0,NaN,Cash,0.58,NaN,Jan-1998,9 years,Scale Technician,764.0,760.0,3000.0,3000.0,A,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RENT,130956066,NaN,w,1.0,2.0,0.0,93.10,7.34,Mar-2018,Nov-2018,764.0,760.0,614.03,May-2018,3000.0,Fully Paid,141.0,NaN,132.0,242.0,18.0,7.0,4.0,26.0,NaN,NaN,7.0,18.0,NaN,7.0,NaN,NaN,0.0,1.0,1.0,4.0,15.0,7.0,6.0,19.0,1.0,7.0,0.0,0.0,0.0,1.0,7.0,0.0,0.0,1.0,2.0,0.0,1.0,NaN,0.00,0.00,NaN,96.7,0.0,1.0,0.0,0.0,major_purchase,n,0.0,141.0,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A4,0.0,36 months,Major purchase,0.0,150592.0,191216.0,30.0,141.0,0.0,30500.0,2.0,0.0,3011.577285,3011.58,11.58,0.0,3000.00,31000.0,https://lendingclub.com/browse/loanDetail.acti...,Source Verified,NaN,2018.0,988xx
421113,0.0,2.0,GA,47.0,55000.0,NaN,Individual,2222.0,10551.0,52.0,0.0,0.0,0.0,N,NaN,NaN,0.0,0.0,NaN,Cash,14.18,NaN,Aug-2001,10+ years,SDO Supervisor,679.0,675.0,5000.0,5000.0,B,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OWN,130968727,63.0,w,0.0,1.0,0.0,166.03,11.98,Mar-2018,Aug-2018,679.0,675.0,5019.97,Apr-2018,5000.0,Fully Paid,4829.0,NaN,77.0,199.0,3.0,3.0,0.0,74.0,74.0,82.0,33.0,3.0,NaN,12.0,NaN,NaN,1.0,3.0,3.0,4.0,7.0,6.0,13.0,18.0,3.0,14.0,0.0,0.0,0.0,2.0,14.0,1.0,1.0,0.0,0.0,2.0,2.0,NaN,0.00,0.00,NaN,95.7,33.3,1.0,1.0,1.0,other,n,0.0,11449.0,NaN,33.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B5,0.0,36 months,Other,0.0,28880.0,61551.0,24.0,28880.0,17431.0,22000.0,1.0,27751.0,5013.30666

### Features to drop

'acc_now_delinq','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy','bc_util','chargeoff_within_12_mths','collection_recovery_fee','collections_12_mths_ex_med','debt_settlement_flag','debt_settlement_flag_date','deferral_term','delinq_2yrs','delinq_amnt','desc','disbursement_method','hardship_amount','hardship_dpd','hardship_end_date','hardship_flag','hardship_last_payment_amount','hardship_length','hardship_loan_status','hardship_payoff_balance_amount','hardship_reason','hardship_start_date','hardship_status','hardship_type','id','il_util','inq_fi','inq_last_12m','inq_last_6mths','last_credit_pull_d','last_fico_range_high','last_fico_range_low','last_pymnt_amnt','last_pymnt_d','max_bal_bc','member_id','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mths_since_last_delinq',
'mths_since_last_major_derog','mths_since_last_record','mths_since_rcnt_il','mths_since_recent_bc','mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','next_pymnt_d','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m','open_acc_6m','open_act_il','open_il_12m','open_il_24m','open_rv_12m','open_rv_24m','orig_projected_additional_accrued_interest','out_prncp','out_prncp_inv','payment_plan_start_date','pct_tl_nvr_dlq','percent_bc_gt_75','policy_code','pub_rec','pub_rec_bankruptcies','recoveries','revol_bal','revol_bal_joint','revol_util','sec_app_chargeoff_within_12_mths','sec_app_collections_12_mths_ex_med','sec_app_earliest_cr_line','sec_app_inq_last_6mths','sec_app_mths_since_last_major_derog','sec_app_num_rev_accts','sec_app_open_act_il','sec_app_revol_util','settlement_amount','settlement_date','settlement_percentage','settlement_status','settlement_term','tax_liens','tot_coll_amt','tot_cur_bal','tot_hi_cred_lim','total_acc','total_bal_ex_mort',
'total_bal_il','total_bc_limit','total_cu_tl','total_il_high_credit_limit','total_pymnt','total_pymnt_inv','total_rec_int','total_rec_late_fee','total_rec_prncp','total_rev_hi_lim','url'

### Features to use ( Features that can be known at the time of loan issuance)

In [261]:
accepted_new = accepted[['addr_state','all_util','annual_inc','annual_inc_joint','application_type',
'dti','dti_joint','earliest_cr_line','emp_length','fico_range_high',
'fico_range_low','funded_amnt','funded_amnt_inv','grade','home_ownership',
'initial_list_status','installment','int_rate','issue_d','year',
'loan_amnt','loan_status','mort_acc','purpose','pymnt_plan',
'sec_app_fico_range_high','sec_app_fico_range_low','sec_app_mort_acc',
'sec_app_open_acc','open_acc','sub_grade','term','verification_status',
'verification_status_joint']]

## Data cleaning

In [262]:
display(accepted_new.isnull().sum())

addr_state                        0
all_util                        114
annual_inc                        0
annual_inc_joint             520922
application_type                  0
dti                             410
dti_joint                    520924
earliest_cr_line                  0
emp_length                    38709
fico_range_high                   0
fico_range_low                    0
funded_amnt                       0
funded_amnt_inv                   0
grade                             0
home_ownership                    0
initial_list_status               0
installment                       0
int_rate                          0
issue_d                           0
year                              0
loan_amnt                         0
loan_status                       0
mort_acc                          0
purpose                           0
pymnt_plan                        0
sec_app_fico_range_high      528011
sec_app_fico_range_low       528011
sec_app_mort_acc            

In [263]:
accepted_new.std()

all_util                      20.958029
annual_inc                 77930.145073
annual_inc_joint           60045.208084
dti                           14.740686
dti_joint                      7.756857
fico_range_high               33.390927
fico_range_low                33.390074
funded_amnt                 9283.481052
funded_amnt_inv             9281.158002
installment                  283.120402
int_rate                       5.329718
year                           0.700987
loan_amnt                   9283.481052
mort_acc                       1.840516
sec_app_fico_range_high       45.641146
sec_app_fico_range_low        45.641146
sec_app_mort_acc               1.812359
sec_app_open_acc               6.555835
open_acc                       5.769080
dtype: float64

Address State

In [264]:
#dropping state
accepted_new.drop(labels='addr_state',axis=1, inplace=True)

all_util

In [265]:
accepted_new.all_util.isna().sum() / len(accepted_new.index)
accepted_new['all_util'].dropna(inplace=True)

In [266]:
accepted_new.all_util.isna().sum()

0

annual_inc & annual_inc_joint

In [267]:
#no missing values, taking the max of individual and joint income

accepted_new['annual_inc'] = accepted_new[['annual_inc','annual_inc_joint']].max(axis=1)
accepted_new[accepted_new['application_type']=='Joint App'].head()

,all_util,annual_inc,annual_inc_joint,application_type,dti,dti_joint,earliest_cr_line,emp_length,fico_range_high,fico_range_low,funded_amnt,funded_amnt_inv,grade,home_ownership,initial_list_status,installment,int_rate,issue_d,year,loan_amnt,loan_status,mort_acc,purpose,pymnt_plan,sec_app_fico_range_high,sec_app_fico_range_low,sec_app_mort_acc,sec_app_open_acc,open_acc,sub_grade,term,verification_status,verification_status_joint
421212,80.0,97000.0,97000.0,Joint App,53.03,21.04,Aug-2006,3 years,684.0,680.0,25000.0,25000.0,C,MORTGAGE,w,582.62,14.07,Mar-2018,2018.0,25000.0,Late,2.0,debt_consolidation,n,679.0,675.0,3.0,9.0,8.0,C3,60 months,Verified,Verified
421222,86.0,122000.0,122000.0,Joint App,10.56,7.47,Mar-2008,2 years,709.0,705.0,1000.0,1000.0,D,RENT,w,35.89,17.47,Mar-2018,2018.0,1000.0,Fully Paid,0.0,credit_card,n,639.0,635.0,1.0,13.0,5.0,D1,36 months,Verified,Verified
421291,85.0,145000.0,145000.0,Joint App,43.63,32.01,Nov-2008,4 years,689.0,685.0,20000.0,20000.0,F,MORTGAGE,w,655.08,30.65,Mar-2018,2018.0,20000.0,Charged Off,1.0,credit_card,n,674.0,670.0,3.0,17.0,18.0,F4,60 months,Verified,Verified
421307,93.0,114000.0,114000.0,Joint App,20.79,21.61,May-2012,2 years,709.0,705.0,6000.0,6000.0,D,RENT,w,224.18,20.39,Mar-2018,2018.0,6000.0,Charged Off,0.0,home_improvement,n,634.0,630.0,0.0,9.0,9.0,D4,36 months,Not Verified,Not Verified
421324,48.0,258000.0,258000.0,Joint App,62.70,30.04,Dec-2002,7 years,699.0,695.0,36250.0,36250.0,D,MORTGAGE,w,1336.50,19.42,Mar-2018,2018.0,36250.0,Late,0.0,debt_consolidation,n,689.0,685.0,4.0,20.0,36.0,D3,36 months,Source Verified,Source Verified


Take the MAX of individual and joint income

Application Type

In [268]:
#no missing values... dummifying

pd.get_dummies(accepted_new['application_type'],drop_first=True)

,Joint App
421101,0
421113,0
421120,0
421121,0
421135,0
...,...
2260688,0
2260690,0
2260691,0
2260692,0


Debt to Income Ratio

In [269]:
#dropping nan and negative values

accepted_new['dti'] = accepted_new['dti'].apply(lambda x: np.where(x<0,np.nan,x))
accepted_new.dti.dropna(inplace=True)

In [270]:
#taking the max between dti and dti_joint
accepted_new['dti'] = accepted_new[['dti','dti_joint']].min(axis=1)

In [271]:
#no missing values but have to convert to date time object
accepted_new['earliest_cr_line'] = accepted_new['earliest_cr_line'].str.split("-")
accepted_new['year_since_earliest_crline'] = accepted_new['earliest_cr_line'].apply(lambda x: 2018 - int(x[1])) 
accepted_new.drop(labels='earliest_cr_line', axis=1)
accepted_new.head()

,all_util,annual_inc,annual_inc_joint,application_type,dti,dti_joint,earliest_cr_line,emp_length,fico_range_high,fico_range_low,funded_amnt,funded_amnt_inv,grade,home_ownership,initial_list_status,installment,int_rate,issue_d,year,loan_amnt,loan_status,mort_acc,purpose,pymnt_plan,sec_app_fico_range_high,sec_app_fico_range_low,sec_app_mort_acc,sec_app_open_acc,open_acc,sub_grade,term,verification_status,verification_status_joint,year_since_earliest_crline
421101,1.0,52000.0,NaN,Individual,0.58,NaN,"[Jan, 1998]",9 years,764.0,760.0,3000.0,3000.0,A,RENT,w,93.10,7.34,Mar-2018,2018.0,3000.0,Fully Paid,4.0,major_purchase,n,NaN,NaN,NaN,NaN,7.0,A4,36 months,Source Verified,NaN,20
421113,47.0,55000.0,NaN,Individual,14.18,NaN,"[Aug, 2001]",10+ years,679.0,675.0,5000.0,5000.0,B,OWN,w,166.03,11.98,Mar-2018,2018.0,5000.0,Fully Paid,0.0,other,n,NaN,NaN,NaN,NaN,14.0,B5,36 months,Not Verified,NaN,17
421120,90.0,40000.0,NaN,Individual,20.25,NaN,"[Mar, 2007]",< 1 year,699.0,695.0,7000.0,7000.0,B,MORTGAGE,w,232.44,11.98,Mar-2018,2018.0,7000.0,Fully Paid,0.0,home_improvement,n,NaN,NaN,NaN,NaN,13.0,B5,36 months,Verified,NaN,11
421121,69.0,55000.0,NaN,Individual,26.63,NaN,"[Nov, 2007]",< 1 year,709.0,705.0,20300.0,20300.0,D,RENT,w,509.66,17.47,Mar-2018,2018.0,20300.0,Late,0.0,credit_card,n,NaN,NaN,NaN,NaN,10.0,D1,60 months,Verified,NaN,11
421135,69.0,57000.0,NaN,Individual,27.58,NaN,"[Apr, 2000]",10+ years,684.0,680.0,30000.0,30000.0,D,OWN,w,1143.39,21.85,Mar-2018,2018.0,30000.0,Fully Paid,2.0,debt_consolidation,n,NaN,NaN,NaN,NaN,11.0,D5,36 months,Verified,NaN,18


Employment Length

In [272]:
# grouping the data
accepted_new['emp_length'] = accepted_new['emp_length'].astype(str).str.replace(' years','')
accepted_new['emp_length'] = accepted_new['emp_length'].astype(str).str.replace(' year','')
accepted_new['emp_length'] = accepted_new['emp_length'].astype(str).str.replace('+','')
accepted_new['emp_length'] = accepted_new['emp_length'].astype(str).str.replace('<1','0')
accepted_new['emp_length'] = accepted_new['emp_length'].apply(lambda x: np.where((x =='< 1'),'less than one',x))
accepted_new['emp_length'] = accepted_new['emp_length'].apply(lambda x: np.where((x =='1') | (x=='2') | (x=='3') | (x=='4') | (x=='5'),'one to five',x))
accepted_new['emp_length'] = accepted_new['emp_length'].apply(lambda x: np.where((x =='6') | (x=='7') | (x=='8') | (x=='9'),'six to nine',x))
accepted_new['emp_length'] = accepted_new['emp_length'].apply(lambda x: np.where(x =='10','ten plus',x))

In [273]:
accepted_new['emp_length'] = accepted_new['emp_length'].apply(lambda x: np.where(x=='nan',np.nan,x))
accepted_new.dti.dropna(inplace=True)

#for some reason I can't get rid of the string 'nan' which is 
#what the "missing values" have

In [274]:
accepted_new.emp_length.value_counts()

one to five      199893
ten plus         183246
six to nine       83126
less than one     45123
nan               38709
Name: emp_length, dtype: int64

FICO RANGE

In [275]:
#creating an average FICO Score like we talked about last time
ficos = accepted_new.loc[:,'fico_range_high':'fico_range_low']
accepted_new['average_fico'] = ficos.mean(axis=1)
accepted_new.head()

,all_util,annual_inc,annual_inc_joint,application_type,dti,dti_joint,earliest_cr_line,emp_length,fico_range_high,fico_range_low,funded_amnt,funded_amnt_inv,grade,home_ownership,initial_list_status,installment,int_rate,issue_d,year,loan_amnt,loan_status,mort_acc,purpose,pymnt_plan,sec_app_fico_range_high,sec_app_fico_range_low,sec_app_mort_acc,sec_app_open_acc,open_acc,sub_grade,term,verification_status,verification_status_joint,year_since_earliest_crline,average_fico
421101,1.0,52000.0,NaN,Individual,0.58,NaN,"[Jan, 1998]",six to nine,764.0,760.0,3000.0,3000.0,A,RENT,w,93.10,7.34,Mar-2018,2018.0,3000.0,Fully Paid,4.0,major_purchase,n,NaN,NaN,NaN,NaN,7.0,A4,36 months,Source Verified,NaN,20,762.0
421113,47.0,55000.0,NaN,Individual,14.18,NaN,"[Aug, 2001]",ten plus,679.0,675.0,5000.0,5000.0,B,OWN,w,166.03,11.98,Mar-2018,2018.0,5000.0,Fully Paid,0.0,other,n,NaN,NaN,NaN,NaN,14.0,B5,36 months,Not Verified,NaN,17,677.0
421120,90.0,40000.0,NaN,Individual,20.25,NaN,"[Mar, 2007]",less than one,699.0,695.0,7000.0,7000.0,B,MORTGAGE,w,232.44,11.98,Mar-2018,2018.0,7000.0,Fully Paid,0.0,home_improvement,n,NaN,NaN,NaN,NaN,13.0,B5,36 months,Verified,NaN,11,697.0
421121,69.0,55000.0,NaN,Individual,26.63,NaN,"[Nov, 2007]",less than one,709.0,705.0,20300.0,20300.0,D,RENT,w,509.66,17.47,Mar-2018,2018.0,20300.0,Late,0.0,credit_card,n,NaN,NaN,NaN,NaN,10.0,D1,60 months,Verified,NaN,11,707.0
421135,69.0,57000.0,NaN,Individual,27.58,NaN,"[Apr, 2000]",ten plus,684.0,680.0,30000.0,30000.0,D,OWN,w,1143.39,21.85,Mar-2018,2018.0,30000.0,Fully Paid,2.0,debt_consolidation,n,NaN,NaN,NaN,NaN,11.0,D5,36 months,Verified,NaN,18,682.0
